In [ ]:
from typing import Optional, Dict
from fmbase.source.merra2.model import YearMonth, load_batch
from fmgraphcast.config import config_files
from fmbase.plot.mpl import plot_data
import hydra
import xarray as xa
import  time
from fmbase.util.config import configure, cfg

In [ ]:
hydra.initialize( version_base=None, config_path="../config" )
configure(  'explore-era5' )
t0 = time.time()

def parse_file_parts(file_name):
	return dict(part.split("-", 1) for part in file_name.split("_"))

def dtypes( d: Dict ):
	return { k: type(v) for k,v in d.items() }

res,levels,steps = cfg().model.res,  cfg().model.levels,  cfg().model.steps
year, month, day =  cfg().model.year,  cfg().model.month,  cfg().model.day
train_steps, eval_steps = cfg().task.train_steps, cfg().task.eval_steps
(model_config,task_config) = config_files()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Load MERRA2 Data
#-----------------

params: Dict[str,Dict] = None
state = {}
dts         = cfg().task.data_timestep
start = YearMonth(year,month)
end = YearMonth(year,month+1)
target_lead_times = [ f"{iS*dts}h" for iS in range(1,train_steps+1) ]
eval_lead_times =   [ f"{iS*dts}h" for iS in range(1,eval_steps+1) ]

In [ ]:
print( "  --------------------- MERRA2 ---------------------")
example_batch: xa.Dataset = load_batch( start, end, cfg().task )

print("Loaded Batch:")
for vname, dvar in example_batch.data_vars.items():
	print( f" {vname}{list(dvar.dims)}: shape={dvar.shape}")

In [ ]:
plot_data( {"":example_batch}, f"MERRA2 {month}/{year}" )